Imports for data manipulation and Streaming of Cocktail DB with API

In [36]:
import numpy as np
import pandas as pd
import requests
import string

Based on method seen on: https://github.com/OzanGenc/CocktailAnalysis/blob/main/CocktailDbStreaming.ipynb

In [37]:
#The first 36 caharacters are the list of alphanumerical
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [38]:
def create_url_list():

  url_list = []
  main_url = 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f='

  #for i in (string.printable[:36]):
  for i in (string.printable):

    url_list.append(main_url+i)

  return url_list

In [39]:
url_list = create_url_list()

In [40]:
def get_ingredients(cocktail_dict):

  ingredient = cocktail_dict['strIngredient1']
  i = 2

  ingredient_list = ""

  while ingredient:

    ingredient_list = ingredient + ", " + ingredient_list
    ingredient = cocktail_dict['strIngredient'+str(i)]
  
    i = i+1

  return ingredient_list

In [41]:
cocktails_list = []

for url in url_list:

  try:

    r = requests.get(url)

    if r.json()['drinks']:

      for cocktail_dict in r.json()['drinks']: 

        cocktail = {}
        cocktail['Cocktail Name'] = cocktail_dict['strDrink']
        cocktail['Ingredients'] = get_ingredients(cocktail_dict)
        cocktail['Preparation'] = cocktail_dict['strInstructions']

        cocktails_list.append(cocktail)

  except:
      pass

In [42]:
cocktail_df = pd.DataFrame(cocktails_list)

In [43]:
cocktail_df.drop_duplicates(inplace=True)

In [44]:
cocktail_df.to_csv('cocktails_db.csv', index=False)

## Data exploration
We put our 3 datasets into 3 different dataframes that we will try to merge into one.
Sources:
- data_cocktails.csv : https://www.kaggle.com/datasets/svetlanagruzdeva/cocktails-data
- hotaling_cocktails.csv : https://www.kaggle.com/datasets/shuyangli94/cocktails-hotaling-co
- cocktails_db.csv : https://www.thecocktaildb.com/

In [45]:
cocktails = pd.read_csv('data_cocktails.csv', index_col=0)
cocktails_h = pd.read_csv('hotaling_cocktails.csv')
cocktails_db = pd.read_csv('cocktails_db.csv')

In [46]:
cocktails.head(5)

,strDrink,strCategory,strGlass,strIngredients,Alc_type,Basic_taste,strInstructions,strMeasures,Value_ml,Value_gr,Garnish_amount,Garnish_type
0,'57 Chevy with a White License Plate,Cocktail,Highball glass,Creme De Cacao White,Creamy Liqueur,NaN,1. Fill a rocks glass with ice 2.add white cre...,1 oz white,30.0,NaN,NaN,NaN
1,'57 Chevy with a White License Plate,Cocktail,Highball glass,Vodka,Vodka,NaN,1. Fill a rocks glass with ice 2.add white cre...,1 oz,30.0,NaN,NaN,NaN
2,1-900-FUK-MEUP,Shot,Old-fashioned glass,Grand Marnier,Triple Sec,NaN,Shake ingredients in a mixing tin filled with ...,1/4 oz,7.5,NaN,NaN,NaN
3,1-900-FUK-MEUP,Shot,Old-fashioned glass,Midori Melon Liqueur,Sweet Liqueur,NaN,Shake ingredients in a mixing tin filled with ...,1/4 oz,7.5,NaN,NaN,NaN
4,1-900-FUK-MEUP,Shot,Old-fashioned glass,Malibu Rum,Rum,NaN,Shake ingredients in a mixing tin filled with ...,1/4 oz,7.5,NaN,NaN,NaN


In [47]:
cocktails_h.shape

(687, 9)

In [48]:
cocktails_db

,Cocktail Name,Ingredients,Preparation
0,155 Belmont,"Orange juice, Vodka, Light rum, Dark rum,",Blend with ice. Serve in a wine glass. Garnish...
1,1-900-FUK-MEUP,"Pineapple juice, Cranberry juice, Amaretto, Ma...",Shake ingredients in a mixing tin filled with ...
2,110 in the shade,"Tequila, Lager,",Drop shooter in glass. Fill with beer
3,151 Florida Bushwacker,"Vanilla ice-cream, Coconut liqueur, Milk, Coin...",Combine all ingredients. Blend until smooth. G...
4,252,"Wild Turkey, 151 proof rum,","Add both ingredients to shot glass, shoot, and..."
...,...,...,...
436,Zima Blaster,"Chambord raspberry liqueur, Zima,","Fill glass with ice. Pour in Chambord, then fi..."
437,Zizi Coin-coin,"Lemon, Ice, Lemon juice, Cointreau,","Pour 5cl of Cointreau on ice, add 2cl of fresh..."
438,Zimadori Zinger,"Zima, Midori melon liqueur,",Pour Zima in a collins glass over ice and then...
439,Zippy's Revenge,"Kool-Aid, Rum, Amaretto,",Mix Kool-Aid to taste then add Rum and ammaret...


The goal is to merge all three in a **unique** dataset.

We need to perform several cleaning operations and transformations. Especially deduplication.

#### 1. cocktails

In [49]:
cocktails = cocktails.drop(columns=['strGlass', 'Alc_type', 'Basic_taste', 'strMeasures', 'strMeasures', 
                                    'Value_ml', 'Value_gr', 'Garnish_amount', 'Garnish_type'])
cocktails

,strDrink,strCategory,strIngredients,strInstructions
0,'57 Chevy with a White License Plate,Cocktail,Creme De Cacao White,1. Fill a rocks glass with ice 2.add white cre...
1,'57 Chevy with a White License Plate,Cocktail,Vodka,1. Fill a rocks glass with ice 2.add white cre...
2,1-900-FUK-MEUP,Shot,Grand Marnier,Shake ingredients in a mixing tin filled with ...
3,1-900-FUK-MEUP,Shot,Midori Melon Liqueur,Shake ingredients in a mixing tin filled with ...
4,1-900-FUK-MEUP,Shot,Malibu Rum,Shake ingredients in a mixing tin filled with ...
...,...,...,...,...
1776,Zorbatini,Cocktail,Vodka,Prepare like a Martini. Garnish with a green o...
1777,Zorbatini,Cocktail,Ouzo,Prepare like a Martini. Garnish with a green o...
1778,Zorro,Coffee / Tea,Sambuca,add all and pour black coffee and add whipped ...
1779,Zorro,Coffee / Tea,Bailey'S Irish Cream,add all and pour black coffee and add whipped ...


Main problem : grouping all ingredients into one unique Drink recipe row, like the other datasets

In [50]:
data_cocktails_new = cocktails.groupby('strDrink').agg(strCategory = ('strCategory', 'max'),
                                                    strIngredients =('strIngredients', ', '.join),
                                                    strInstructions = ('strInstructions', 'max')).reset_index()
data_cocktails_new

,strDrink,strCategory,strIngredients,strInstructions
0,'57 Chevy with a White License Plate,Cocktail,"Creme De Cacao White, Vodka",1. Fill a rocks glass with ice 2.add white cre...
1,1-900-FUK-MEUP,Shot,"Grand Marnier, Midori Melon Liqueur, Malibu Ru...",Shake ingredients in a mixing tin filled with ...
2,110 in the shade,Beer,"Tequila, Lager",Drop shooter in glass. Fill with beer
3,151 Florida Bushwacker,Milk / Float / Shake,"Dark Creme De Cacao, Coconut Liqueur, Light Ru...",Combine all ingredients. Blend until smooth. G...
4,155 Belmont,Cocktail,"Light Rum, Orange Juice, Vodka, Dark Rum",Blend with ice. Serve in a wine glass. Garnish...
...,...,...,...,...
468,Zipperhead,Shot,"Chambord Raspberry Liqueur, Soda Water, Vodka","Fill glass with rocks, add straw before puttin..."
469,Zippy's Revenge,Cocktail,"Rum, Kool-Aid Grape, Amaretto",Mix Kool-Aid to taste then add Rum and ammaret...
470,Zizi Coin-coin,Punch / Party Drink,"Cointreau, Lemon Juice, Lemon","Pour 5cl of Cointreau on ice, add 2cl of fresh..."
471,Zorbatini,Cocktail,"Vodka, Ouzo",Prepare like a Martini. Garnish with a green o...


Let's take only cocktails, filter out the rest.
And then, input it into our future complete dataset.

In [51]:
data_cocktails_new = data_cocktails_new[data_cocktails_new['strCategory'] == "Cocktail"]
full_dataset = data_cocktails_new.drop(columns=['strCategory'])

In [52]:
full_dataset.rename(columns={'strDrink':"Cocktail", 'strIngredients':"Ingredients", 'strInstructions':"Preparation"}, inplace=True)

In [53]:
full_dataset.shape

(58, 3)

#### 2. cocktails_h

In [54]:
cocktails_h.head(5)

,Cocktail_Name,Bartender,Bar/Company,Location,Ingredients,Garnish,Glassware,Preparation,Notes
0,Flor de Amaras,Kelly McCarthy,NaN,Boston,"1.5 oz Mezcal, 1 oz Hibiscus Simple Syrup*, .5...",Marigold Petals,NaN,*Hibiscus Simple Syrup:\n1:1 w/ a cup of dried...,NaN
1,The Happy Place,Elizabeth Montana,Forgery & Verso,San Francisco,"2 oz Junipero Gin, .75 oz House-made Cranberry...","Dehydrated Lemon Wheel, Sprig of Rosemary",NaN,*House-made Cranberry syrup: \n-- 2 cups Fresh...,Junipero Gin 20th Anniversary Signature Cocktail
2,Bon Voyage Pisco Punch,Jon Morales,NaN,San Francisco,"1500 ml BarSol Selecto Italia Pisco, 750 ml Le...",NaN,Punch Bowl,*Pineapple Gomme: \nMix equal parts (1.5 cups)...,NaN
3,Still Life of a Pineapple,Daniel Braganca,Backbar,Somerville,"1.5 oz BarSol Primero Quebranta Pisco, .75 oz ...",NaN,NaN,*Pineapple Syrup:\n<em>Equal parts pineapple b...,NaN
4,The Bittered Valley,Nik Virrey,NaN,Seattle,"1.25 oz Luxardo Maraschino Liqueur, 4 drops Ac...",NaN,NaN,"1st glass ingredients:\nLuxardo Maraschino, Ac...",NaN


In [55]:
cocktails_h = cocktails_h.drop(columns=['Bartender', 'Garnish', 'Glassware', 'Notes'])
cocktails_h.tail(5)

,Cocktail_Name,Bar/Company,Location,Ingredients,Preparation
682,The Negroni Bianco,NaN,NaN,"1.5 oz Luxardo Bitter Bianco, .75 oz Tempus Fu...",NaN
683,Match Grip Julep,Death & Co.,New York,"1.5 oz H by HINE, .5 oz Jamaican Rum, 1 tsp Ca...",NaN
684,The Dry Cruise,"Rob Roy, The Hideout",Seattle,"1 oz Luxardo Bitter Bianco, 1 oz Lustau Oloros...",NaN
685,Red Hook,Henrietta Red,Nashville,"2 oz Rye Whiskey, 5 oz Punt e Mes, .5 oz Luxar...",NaN
686,Union Jack,Union Lodge No.1,Denver,".75 oz Luxardo Maraschino Liqueur, .75 oz Gree...",NaN


Get rid of dosage of ingredients to have consistent data: only ingredients.
Regex tool used: https://regexr.com/

In [56]:
cocktails_h = cocktails_h.replace({'Preparation': r'\n'}, {'Preparation': ' '}, regex=True)
cocktails_h = cocktails_h.replace({'Ingredients': r'(.?[0-9].?\w*) [oz|ml|gr|drops|tsp]+ '}, {'Ingredients': ' '}, regex=True)

In [57]:
cocktails_h.rename(columns={'Cocktail_Name':"Cocktail"}, inplace=True)

In [58]:
full_dataset_new = pd.concat([full_dataset, cocktails_h], axis=0 ).reset_index()
full_dataset_new.drop(columns=['index'], inplace=True)
full_dataset_new

,Cocktail,Ingredients,Preparation,Bar/Company,Location
0,'57 Chevy with a White License Plate,"Creme De Cacao White, Vodka",1. Fill a rocks glass with ice 2.add white cre...,NaN,NaN
1,155 Belmont,"Light Rum, Orange Juice, Vodka, Dark Rum",Blend with ice. Serve in a wine glass. Garnish...,NaN,NaN
2,9 1/2 Weeks,"Orange Curacao, Strawberry Liqueur, Orange Jui...",Combine all ingredients in glass mixer. Chill ...,NaN,NaN
3,A Gilligan's Island,"Peach Schnapps, Cranberry Juice, Orange Juice,...","Shaken, not stirred!",NaN,NaN
4,A True Amaretto Sour,"Lemon, Maraschino Cherry, Amaretto",Rub the rim of an old fashioned glass with lem...,NaN,NaN
...,...,...,...,...,...
740,The Negroni Bianco,"Luxardo Bitter Bianco, Tempus Fugit Alessio ...",NaN,NaN,NaN
741,Match Grip Julep,"H by HINE, Jamaican Rum, Cacao, Branca Menta...",NaN,Death & Co.,New York
742,The Dry Cruise,"Luxardo Bitter Bianco, Lustau Oloroso Sherry,...",NaN,"Rob Roy, The Hideout",Seattle
743,Red Hook,"Rye Whiskey, Punt e Mes, Luxardo Maraschino ...",NaN,Henrietta Red,Nashville


In [59]:
full_dataset_new.drop_duplicates(subset="Cocktail", inplace=True)
full_dataset_new['Cocktail'].describe()

count                                      739
unique                                     739
top       '57 Chevy with a White License Plate
freq                                         1
Name: Cocktail, dtype: object

### Create final dataset by combination

In [60]:
cocktails_db.rename(columns={"Cocktail Name": "Cocktail"}, inplace=True)
cocktails_db

,Cocktail,Ingredients,Preparation
0,155 Belmont,"Orange juice, Vodka, Light rum, Dark rum,",Blend with ice. Serve in a wine glass. Garnish...
1,1-900-FUK-MEUP,"Pineapple juice, Cranberry juice, Amaretto, Ma...",Shake ingredients in a mixing tin filled with ...
2,110 in the shade,"Tequila, Lager,",Drop shooter in glass. Fill with beer
3,151 Florida Bushwacker,"Vanilla ice-cream, Coconut liqueur, Milk, Coin...",Combine all ingredients. Blend until smooth. G...
4,252,"Wild Turkey, 151 proof rum,","Add both ingredients to shot glass, shoot, and..."
...,...,...,...
436,Zima Blaster,"Chambord raspberry liqueur, Zima,","Fill glass with ice. Pour in Chambord, then fi..."
437,Zizi Coin-coin,"Lemon, Ice, Lemon juice, Cointreau,","Pour 5cl of Cointreau on ice, add 2cl of fresh..."
438,Zimadori Zinger,"Zima, Midori melon liqueur,",Pour Zima in a collins glass over ice and then...
439,Zippy's Revenge,"Kool-Aid, Rum, Amaretto,",Mix Kool-Aid to taste then add Rum and ammaret...


In [61]:
final_df = pd.concat([full_dataset_new, cocktails_db], axis=0).reset_index()

In [62]:
final_df.drop_duplicates(subset="Cocktail", inplace=True)
final_df['Cocktail'].describe()

count                                     1129
unique                                    1129
top       '57 Chevy with a White License Plate
freq                                         1
Name: Cocktail, dtype: object

In [63]:
final_df.drop(columns=['index'], inplace=True)
final_df

,Cocktail,Ingredients,Preparation,Bar/Company,Location
0,'57 Chevy with a White License Plate,"Creme De Cacao White, Vodka",1. Fill a rocks glass with ice 2.add white cre...,NaN,NaN
1,155 Belmont,"Light Rum, Orange Juice, Vodka, Dark Rum",Blend with ice. Serve in a wine glass. Garnish...,NaN,NaN
2,9 1/2 Weeks,"Orange Curacao, Strawberry Liqueur, Orange Jui...",Combine all ingredients in glass mixer. Chill ...,NaN,NaN
3,A Gilligan's Island,"Peach Schnapps, Cranberry Juice, Orange Juice,...","Shaken, not stirred!",NaN,NaN
4,A True Amaretto Sour,"Lemon, Maraschino Cherry, Amaretto",Rub the rim of an old fashioned glass with lem...,NaN,NaN
...,...,...,...,...,...
1174,Zipperhead,"Soda water, Vodka, Chambord raspberry liqueur,","Fill glass with rocks, add straw before puttin...",NaN,NaN
1175,Zima Blaster,"Chambord raspberry liqueur, Zima,","Fill glass with ice. Pour in Chambord, then fi...",NaN,NaN
1176,Zizi Coin-coin,"Lemon, Ice, Lemon juice, Cointreau,","Pour 5cl of Cointreau on ice, add 2cl of fresh...",NaN,NaN
1177,Zimadori Zinger,"Zima, Midori melon liqueur,",Pour Zima in a collins glass over ice and then...,NaN,NaN


In [64]:
final_df = final_df.replace({'Ingredients': r'(.?[0-9].?\w*) [oz|ml|gr|drops|tsp|dash|c|float|bsp]+ '}, {'Ingredients': ' '}, regex=True)

In [65]:
final_df['Cocktail'].str.strip()
final_df['Ingredients'].str.strip(', ')
final_df['Preparation'].str.strip()

0       1. Fill a rocks glass with ice 2.add white cre...
1       Blend with ice. Serve in a wine glass. Garnish...
2       Combine all ingredients in glass mixer. Chill ...
3                                    Shaken, not stirred!
4       Rub the rim of an old fashioned glass with lem...
                              ...                        
1174    Fill glass with rocks, add straw before puttin...
1175    Fill glass with ice. Pour in Chambord, then fi...
1176    Pour 5cl of Cointreau on ice, add 2cl of fresh...
1177    Pour Zima in a collins glass over ice and then...
1179    Serve without ice. At least the juice shold ha...
Name: Preparation, Length: 1129, dtype: object

In [66]:
new_row = pd.Series({'Cocktail': "Bloody Mary", 'Ingredients': "Lime, Tabasco sauce, Worcestershire sauce, Lemon juice, Tomato juice, Vodka",
                        'Preparation' : "Stirring gently, pour all ingredients into highball glass. Garnish."})
final_df = pd.concat([final_df, new_row.to_frame().T], ignore_index=True)

#### Export final dataset

In [67]:
final_df.to_csv('final_cocktails.csv', encoding='UTF-8', index=False)